In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

# Exercises

### Create a spark data frame that contains your favorite programming languages.

The name of the column should be language

In [6]:
import pandas as pd

pd_df = pd.DataFrame({'language': ['python', 'R', 'java', 'c++', 'javascript', 'julia']}, 
                     index = [1, 2, 3, 4, 5, 6])

pd_df

,language
1,python
2,R
3,java
4,c++
5,javascript
6,julia


In [7]:
sp_df = spark.createDataFrame(pd_df)
sp_df.show()

+----------+
|  language|
+----------+
|    python|
|         R|
|      java|
|       c++|
|javascript|
|     julia|
+----------+



View the schema of the dataframe

In [8]:
sp_df.printSchema()

root
 |-- language: string (nullable = true)



Output the shape of the dataframe

In [38]:
print(sp_df.count(), "rows", len(sp_df.columns), "columns")

6 rows 1 columns


Show the first 5 records in the dataframe

In [9]:
sp_df.show(5)

+----------+
|  language|
+----------+
|    python|
|         R|
|      java|
|       c++|
|javascript|
+----------+
only showing top 5 rows



### Load the mpg dataset as a spark dataframe.

In [11]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



Create 1 column of output that contains a message like the one below:

> The 1999 audi a4 has a 4 cylinder engine.

For each vehicle.

In [17]:
import pyspark.sql.functions as F

mpg.select(F.concat(F.lit('The '),
                  mpg.year,
                  F.lit(' '),
                  mpg.manufacturer,
                  F.lit(' '),
                  mpg.model,
                  F.lit(' has a '),
                  mpg.cyl,
                  F.lit(' cylinder engine.')
                 ).alias('output')).show(5, truncate=False)

+-----------------------------------------+
|output                                   |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



Transform the trans column so that it only contains either manual or auto.

In [31]:
mpg.select('trans', F.regexp_extract('trans', r"(\w.*)(\(.*\))", 1).alias('trans_text'),).show(5)

+----------+----------+
|     trans|trans_text|
+----------+----------+
|  auto(l5)|      auto|
|manual(m5)|    manual|
|manual(m6)|    manual|
|  auto(av)|      auto|
|  auto(l5)|      auto|
+----------+----------+
only showing top 5 rows



### Load the tips dataset as a spark dataframe.

In [46]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



What percentage of observations are smokers?

In [102]:
(
    tips.groupBy('smoker')
    .count()
    .withColumn(
        'percent',
        F.concat(
            F.round((F.col('count')/tips.count()*100), 0)
                 .cast('int'),
                 F.lit('%'))
    )
).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|    62%|
|   Yes|   93|    38%|
+------+-----+-------+



Create a column that contains the tip percentage

In [49]:
tips = tips.withColumn(
    "tip_percent", F.expr('ROUND((tip / total_bill) * 100)'))

tips.show(5)

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percent|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        6.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|       16.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|       17.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|       14.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|       15.0|
+----------+----+------+------+---+------+----+-----------+
only showing top 5 rows



### Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [50]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



Convert the temperatures to farenheight.

In [52]:
weather = weather.withColumn(
    "temp_max_f", F.expr('temp_max * 9/5 + 32'))
weather = weather.withColumn(
    "temp_min_f", F.expr('temp_min * 9/5 + 32'))

weather.show(5)

+----------+-------------+--------+--------+----+-------+----------+----------+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max_f|temp_min_f|
+----------+-------------+--------+--------+----+-------+----------+----------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|     55.04|      41.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|     51.08|     37.04|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|     53.06|     44.96|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|     53.96|     42.08|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|     48.02|     37.04|
+----------+-------------+--------+--------+----+-------+----------+----------+
only showing top 5 rows



Which month has the most rain, on average?

In [62]:
(
    weather.withColumn("month", F.month("date"))
    .groupBy("month")
    .agg(F.avg("precipitation").alias("avg_rain"))
    .sort(F.desc("avg_rain"))
    .show(5)
)

+-----+------------------+
|month|          avg_rain|
+-----+------------------+
|   11| 5.354166666666667|
|   12| 5.021774193548388|
|    3| 4.888709677419355|
|   10| 4.059677419354839|
|    1|3.7580645161290316|
+-----+------------------+
only showing top 5 rows



Which year was the windiest?

In [63]:
(
    weather.withColumn("year", F.year("date"))
    .groupBy("year")
    .agg(F.avg("wind").alias("avg_wind"))
    .sort(F.desc("avg_wind"))
    .show(5)
)

+----+------------------+
|year|          avg_wind|
+----+------------------+
|2012| 3.400819672131147|
|2014|3.3876712328767136|
|2015|  3.15972602739726|
|2013|3.0158904109589044|
+----+------------------+



What is the most frequent type of weather in January?

In [104]:
(weather
 .withColumn('month', F.month('date'))
 .filter(F.col('month') == 1)
 .groupBy('weather')
 .count()
 .sort(F.col('count').desc())
 .show()
)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



What is the average high and low tempurature on sunny days in July in 2013 and 2014?

In [108]:
(weather
 .filter(F.month('date') == 7)
 .filter(F.year('date') > 2012)
 .filter(F.year('date') < 2015)
 .filter(F.col('weather') == F.lit('sun'))
 .agg(F.avg('temp_max_f').alias('average_high_temp'), F.avg('temp_min_f').alias('average_low_temp'))
 .show()
)

+-----------------+-----------------+
|average_high_temp| average_low_temp|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



What percentage of days were rainy in q3 of 2015?

In [111]:
(weather
 .filter(F.year('date') == 2015)
 .filter(F.quarter('date') == 3)
 .select(F.when(F.col('weather') == 'rain', 1).otherwise(0).alias('rain'))
 .agg(F.mean('rain'))
 .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



For each year, find what percentage of days it rained (had non-zero precipitation).

In [112]:
(weather
 .filter(F.year('date') == 2015)
 .filter(F.quarter('date') == 3)
 .select(F.when(F.col('precipitation') > 0, 1).otherwise(0).alias('rain'))
 .agg(F.mean('rain'))
 .show()
)

+-------------------+
|          avg(rain)|
+-------------------+
|0.18478260869565216|
+-------------------+

